In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/
!cp /content/drive/MyDrive/nlp-final-project/dontpatronizeme_categories.tsv ./
!cp /content/drive/MyDrive/nlp-final-project/dontpatronizeme_pcl.tsv ./
!cp /content/drive/MyDrive/nlp-final-project/README.txt ./

/content


In [3]:
import random
import os
from urllib import request
import pandas as pd
import numpy as np

In [4]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [5]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py


In [6]:
!pwd

/content


In [7]:
dpm = None

In [8]:
from dont_patronize_me import DontPatronizeMe
# Initialize a dpm (Don't Patronize Me) object.
# It takes two areguments as input: 
# (1) Path to the directory containing the training set files, which is the root directory of this notebook.
# (2) Path to the test set, which will be released when the evaluation phase begins. In this example, 
# we use the dataset for Subtask 1, which the code will load without labels.
dpm = DontPatronizeMe('.', 'dontpatronizeme_pcl.tsv')

In [9]:
! git clone https://github.com/Perez-AlmendrosC/dontpatronizeme.git

Cloning into 'dontpatronizeme'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 200 (delta 87), reused 60 (delta 21), pack-reused 0
Receiving objects: 100% (200/200), 1.64 MiB | 5.01 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [10]:
teids = pd.read_csv('/content/dontpatronizeme/semeval-2022/practice splits/dev_semeval_parids-labels.csv')

In [11]:
teids.par_id = teids.par_id.astype(str)

### Load task 2 data

In [12]:
dpm.load_task2()
task2_df = dpm.train_task2_df

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [13]:
task2_df

,par_id,art_id,text,keyword,country,label
0,4046,@@14767805,we also know that they can benefit by receivin...,hopeless,us,"[1, 0, 0, 1, 0, 0, 0]"
1,1279,@@7896098,pope francis washed and kissed the feet of mus...,refugee,ng,"[0, 1, 0, 0, 0, 0, 0]"
2,8330,@@17252299,many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 1, 0, 0, 0, 0]"
3,4063,@@3002894,"""budding chefs , like """" fred """" , """" winston ...",in-need,ie,"[1, 0, 0, 1, 1, 1, 0]"
4,4089,@@25597822,"""in a 90-degree view of his constituency , one...",homeless,pk,"[1, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...
988,873,@@20374243,citing the fact that these kids who died at go...,poor-families,sg,"[1, 0, 0, 0, 1, 0, 0]"
989,10070,@@15573661,fern ? ndez was a well-known philanthropist wh...,disabled,ng,"[1, 0, 0, 0, 0, 0, 0]"
990,6484,@@2559173,"touched much by their plight , commanding offi...",homeless,lk,"[1, 0, 0, 0, 0, 1, 0]"
991,6249,@@1947926,she reiterated her ministry 's commitment to p...,women,gh,"[1, 0, 0, 0, 0, 0, 0]"


### Fine tuning RoBERTa for Classification
[BERT reference using HuggingFace](https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b)

[HuggingFace tutorial for RoBERTa](https://huggingface.co/docs/transformers/master/en/model_doc/roberta#transformers.RobertaForSequenceClassification)

### Installation

In [14]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 72.3 MB/s 
     |████████████████████████████████| 895 kB 73.1 MB/s 
     |████████████████████████████████| 61 kB 515 kB/s 
     |████████████████████████████████| 3.3 MB 69.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [15]:
!pip install datasets

     |████████████████████████████████| 298 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 78.8 MB/s 
     |████████████████████████████████| 132 kB 82.4 MB/s 
     |████████████████████████████████| 243 kB 81.5 MB/s 
     |████████████████████████████████| 192 kB 59.7 MB/s 
     |████████████████████████████████| 160 kB 75.6 MB/s 
     |████████████████████████████████| 271 kB 77.9 MB/s 


In [16]:
task2_df['text'][0]

'we also know that they can benefit by receiving counseling from someone who can help them understand that their feelings are normal and that their situation is not hopeless ; someone who can help them put their situation in perspective and help them communicate with others who could provide support ; someone knowledgeable about resources they can access ; someone who can help them plan for their needs and the needs of their child by developing either a parenting plan or an adoption plan .'

### Fine tuning

In [17]:
from transformers import RobertaConfig, RobertaModel,RobertaForSequenceClassification
configuration = RobertaConfig()
model = RobertaForSequenceClassification(configuration)
configuration = model.config
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", num_labels =7)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 7)
from datasets import Dataset
dataset = Dataset.from_pandas(task2_df)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [18]:
dataset

Dataset({
    features: ['par_id', 'art_id', 'text', 'keyword', 'country', 'label'],
    num_rows: 993
})

In [19]:
len(task2_df)*8/10

794.4

In [20]:
train_ds = Dataset.from_pandas(task2_df[:795])
valid_ds = Dataset.from_pandas(task2_df[795:])
tokenized_train_ds = train_ds.map(lambda x: tokenizer(x["text"],max_length=512, padding="max_length", truncation=True, return_tensors="pt"))
tokenized_valid_ds = valid_ds.map(lambda x: tokenizer(x["text"],max_length=512, padding="max_length", truncation=True, return_tensors="pt"))

  0%|          | 0/795 [00:00<?, ?ex/s]

  0%|          | 0/198 [00:00<?, ?ex/s]

In [21]:
# text = "we find ourselves in this situation because people are living longer and a number of more local factors including that the number of people requiring nursing care in their care home is increasing coupled with the increased demands for council-funded care to vulnerable people within their own homes ."

# tokenizer(text, max_length=256, padding='max_length', return_tensors="pt")

In [22]:
len("we find ourselves in this situation because people are living longer and a number of more local factors including that the number of people requiring nursing care in their care home is increasing coupled with the increased demands for council-funded care to vulnerable people within their own homes .".split(" "))

49

In [23]:
tokenized_train_ds = tokenized_train_ds.rename_column("label", "labels")
tokenized_valid_ds = tokenized_valid_ds.rename_column("label", "labels")
tokenized_train_ds = tokenized_train_ds.remove_columns(['art_id', 'keyword', 'country'])
tokenized_valid_ds = tokenized_valid_ds.remove_columns(['art_id', 'keyword', 'country'])
tokenized_train_ds.set_format("torch")
tokenized_valid_ds.set_format("torch")
tokenized_valid_ds, tokenized_train_ds

(Dataset({
     features: ['par_id', 'text', 'labels', 'input_ids', 'attention_mask'],
     num_rows: 198
 }), Dataset({
     features: ['par_id', 'text', 'labels', 'input_ids', 'attention_mask'],
     num_rows: 795
 }))

In [24]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        #self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [25]:
task2_df.loc[np.argmax( task2_df['text'].str.len())]['text']

'"most are from desperately poor horn of africa countries and like tens of thousands each year , were willing to risk the treacherous journey through war-torn and impoverished yemen in the hope of finding work in saudi arabia and other wealthy gulf arab states . but their plan was not to be . caught and detained by the yemeni authorities , the 600 or so men now await deportation , prevented from leaving their makeshift jail by armed soldiers . conditions at the warehouse are growing increasingly desperate . several days ago , the authorities stopped handing out food and basic supplies . "" i came from djibouti to work . they used to give us small amounts of food . if there is no food , we will die , "" said one migrant who declined to give his name . "" if there is no solution , they will deport us to our countries or get us out of here "" , the man added . yemen , the poorest country on the arabian peninsula , has long been a transit point for migrants and refugees from east africa , 

In [26]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 1e-05

In [27]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=task2_df.sample(frac=train_size,random_state=200)
train_dataset = train_dataset[['par_id', 'text', 'label']]
test_dataset=task2_df.drop(train_dataset.index).reset_index(drop=True)
test_dataset = test_dataset[['par_id', 'text', 'label']]
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(task2_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (993, 6)
TRAIN Dataset: (794, 3)
TEST Dataset: (199, 3)


In [28]:
train_dataset.label

0      [1, 0, 1, 0, 0, 1, 0]
1      [1, 0, 0, 0, 0, 0, 0]
2      [1, 0, 0, 1, 0, 0, 0]
3      [1, 0, 0, 0, 0, 1, 0]
4      [1, 0, 0, 0, 1, 1, 0]
               ...          
789    [1, 0, 0, 0, 1, 1, 0]
790    [1, 0, 0, 0, 0, 0, 0]
791    [0, 0, 0, 1, 0, 0, 0]
792    [1, 0, 0, 0, 0, 0, 0]
793    [0, 0, 1, 1, 1, 1, 0]
Name: label, Length: 794, dtype: object

In [29]:
train_dataset

,par_id,text,label
0,8600,b.c. housing minister selina robinson said tha...,"[1, 0, 1, 0, 0, 1, 0]"
1,9788,turkit 's heartening to see that measures are ...,"[1, 0, 0, 0, 0, 0, 0]"
2,7691,who cares if he gives lots of charity to homel...,"[1, 0, 0, 1, 0, 0, 0]"
3,7332,un app lets you know what life as a refugee is...,"[1, 0, 0, 0, 0, 1, 0]"
4,8260,"""yangon , myanmar , september 29 , 2017 -- san...","[1, 0, 0, 0, 1, 1, 0]"
...,...,...,...
789,8495,he stressed for extricating the poorest from t...,"[1, 0, 0, 0, 1, 1, 0]"
790,8247,""""""" those ( rohingyas ) who somehow entered th...","[1, 0, 0, 0, 0, 0, 0]"
791,9690,"what i have found , especially with the last w...","[0, 0, 0, 1, 0, 0, 0]"
792,5092,""""""" because our democracies are inclusive , we...","[1, 0, 0, 0, 0, 0, 0]"


In [30]:
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [31]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [32]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = transformers.RobertaModel.from_pretrained('roberta-base', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = RobertaClass()
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

In [33]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [34]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)  

In [35]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        targets = targets.unsqueeze(1)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(1):
    train(epoch)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

In [ ]:
import os
%cd /content/drive/MyDrive/ColabNotebooks/output/checkpoint-2000
os.path.exists("/content/drive/MyDrive/ColabNotebooks/output/checkpoint-2000")

/content/drive/MyDrive/ColabNotebooks/output/checkpoint-2000


True

In [ ]:
!mv /content/output /content/drive/MyDrive/Colab\ Notebooks

In [ ]:
dpm.load_test()

In [ ]:
# ----- 3. Predict -----#
# Load test data

X_test = list(test_df["text"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Load trained model
model_path = "/content/drive/MyDrive/ColabNotebooks/output/checkpoint-2000"
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=7)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

loading configuration file /content/drive/MyDrive/ColabNotebooks/output/checkpoint-2000/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /content/drive/MyDrive/ColabNotebooks/output/checkpoint-2000/pytorch_model.bin
All model checkpoint weights were use

In [ ]:
raw_pred

array([[ 4.7972746, -4.5505958],
       [-4.494454 ,  4.6093245],
       [ 5.0250854, -4.861603 ],
       ...,
       [ 1.7144836, -1.0891386],
       [ 4.2896724, -4.0312204],
       [ 4.604078 , -4.2238655]], dtype=float32)

In [ ]:
 pd.DataFrame(y_pred.tolist()).to_csv("/content/drive/MyDrive/ColabNotebooks/output/predictions.tsv", sep="\t", index=False)

In [ ]:
y_pred.tofile("/content/drive/MyDrive/ColabNotebooks/output/ypreds.csv")

In [ ]:
ypreds = [[y] for y in y_pred]
ypreds

[[0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0]

In [ ]:
%cd /content/

/content


In [ ]:
!mkdir ref res

In [ ]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [ ]:
labels2file(test_df.label.apply(lambda x:[x]).tolist(), os.path.join('ref/', 'task1.txt'))

In [ ]:
labels2file(ypreds, os.path.join('res/', 'task1.txt'))

In [ ]:
%cd /content/

/content


In [ ]:
!cp scores.txt /content/drive/MyDrive/ColabNotebooks/output/

cp: cannot stat 'scores.txt': No such file or directory


In [ ]:
!python3 evaluation.py . .

In [ ]:
!cat scores.txt

task1_precision:0.5944055944055944
task1_recall:0.4748603351955307
task1_f1:0.5279503105590062


## Predicting the practice dev split

In [ ]:
len(tedf1.text.tolist())

2094

In [ ]:
# ----- 3. Predict -----#
# Load test data

X_test = list(tedf1["text"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Load trained model
model_path = "/content/drive/MyDrive/ColabNotebooks/output/checkpoint-2000"
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=2)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

loading configuration file /content/drive/MyDrive/ColabNotebooks/output/checkpoint-2000/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /content/drive/MyDrive/ColabNotebooks/output/checkpoint-2000/pytorch_model.bin
All model checkpoint weights were use

In [ ]:
# run predictions
preds_task1, _, _ = test_trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 2094
  Batch size = 8


In [ ]:
# Preprocess raw predictions
y_pred1 = np.argmax(preds_task1, axis=1)

In [ ]:
y_pred1, len(y_pred1)

(array([1, 1, 0, ..., 0, 0, 0]), 2094)

In [ ]:
 pd.DataFrame(y_pred1.tolist()).to_csv("/content/drive/MyDrive/ColabNotebooks/output/predictions_dev.tsv", sep="\t", index=False)

In [ ]:
y_pred1.tofile("/content/drive/MyDrive/ColabNotebooks/output/ypreds_dev.csv")

In [ ]:
ypreds1 = [[y] for y in y_pred1]
ypreds1

[[1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]

In [ ]:
%cd /content/

/content


In [ ]:
!python3 evaluation.py . .

In [ ]:
!cat scores.txt

task1_precision:0.5944055944055944
task1_recall:0.4748603351955307
task1_f1:0.5279503105590062


In [ ]:
task2_df

,par_id,art_id,text,keyword,country,label
0,4046,@@14767805,we also know that they can benefit by receivin...,hopeless,us,"[1, 0, 0, 1, 0, 0, 0]"
1,1279,@@7896098,pope francis washed and kissed the feet of mus...,refugee,ng,"[0, 1, 0, 0, 0, 0, 0]"
2,8330,@@17252299,many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 1, 0, 0, 0, 0]"
3,4063,@@3002894,"""budding chefs , like """" fred """" , """" winston ...",in-need,ie,"[1, 0, 0, 1, 1, 1, 0]"
4,4089,@@25597822,"""in a 90-degree view of his constituency , one...",homeless,pk,"[1, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...
988,873,@@20374243,citing the fact that these kids who died at go...,poor-families,sg,"[1, 0, 0, 0, 1, 0, 0]"
989,10070,@@15573661,fern ? ndez was a well-known philanthropist wh...,disabled,ng,"[1, 0, 0, 0, 0, 0, 0]"
990,6484,@@2559173,"touched much by their plight , commanding offi...",homeless,lk,"[1, 0, 0, 0, 0, 1, 0]"
991,6249,@@1947926,she reiterated her ministry 's commitment to p...,women,gh,"[1, 0, 0, 0, 0, 0, 0]"


In [ ]:
# tf_train_dataset = tokenized_train_ds.remove_columns(["text"])#.with_format("torch")
# tf_eval_dataset = tokenized_valid_ds.remove_columns(["text"])#.with_format("torch")

In [ ]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(tf_train_dataset, shuffle=True, batch_size=16)
# eval_dataloader = DataLoader(tf_eval_dataset, batch_size=16)

In [ ]:
# train_dataloader.dataset[0], train_dataloader.dataset[1]

In [ ]:
# from transformers import get_scheduler

# num_epochs = 3
# num_training_steps = num_epochs * len(train_dataset)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps
# )

In [ ]:
# train_dataloader

In [ ]:
# import torch

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

In [ ]:
# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

# model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataset:
#         print(batch.items())
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)

In [ ]:
# from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

# # Set tokenizer hyperparameters.
# MAX_SEQ_LEN = 256
# BATCH_SIZE = 16
# PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
# UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)


# # Define columns to read.
# label_field = Field(sequential=False, use_vocab=False, batch_first=True)
# text_field = Field(use_vocab=False, 
#                    tokenize=tokenizer.encode, 
#                    include_lengths=False, 
#                    batch_first=True,
#                    fix_length=MAX_SEQ_LEN, 
#                    pad_token=PAD_INDEX, 
#                    unk_token=UNK_INDEX)

In [ ]:
# train_iter, valid_iter = BucketIterator.splits((train_ds, valid_ds),
#                                                batch_size=BATCH_SIZE,
#                                                device=device,
#                                                shuffle=True,
#                                                sort_key=lambda x: len(x.text), 
#                                                sort=True, 
#                                                sort_within_batch=False)
# test_iter = Iterator(test_ds, batch_size=BATCH_SIZE, device=device, train=False, shuffle=False, sort=False)

# for (source, target), _ in train_iter:
#     mask = (source != PAD_INDEX).type(torch.uint8)

#     y_pred = model(input_ids=source,  
#                    attention_mask=mask)

In [ ]:
# # Model with classifier layers on top of RoBERTa
# class ROBERTAClassifier(torch.nn.Module):
#     def __init__(self, dropout_rate=0.3):
#         super(ROBERTAClassifier, self).__init__()
        
#         self.roberta = RobertaModel.from_pretrained('roberta-base')
#         self.d1 = torch.nn.Dropout(dropout_rate)
#         self.l1 = torch.nn.Linear(768, 64)
#         self.bn1 = torch.nn.LayerNorm(64)
#         self.d2 = torch.nn.Dropout(dropout_rate)
#         self.l2 = torch.nn.Linear(64, 2)
        
#     def forward(self, input_ids, attention_mask):
#         _, x = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
#         x = self.d1(x)
#         x = self.l1(x)
#         x = self.bn1(x)
#         x = torch.nn.Tanh()(x)
#         x = self.d2(x)
#         x = self.l2(x)
        
#         return x 

In [ ]:
# # Set tokenizer hyperparameters.
# MAX_SEQ_LEN = 256
# BATCH_SIZE = 16
# PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
# UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)


# # Define columns to read.
# label_field = Field(sequential=False, use_vocab=False, batch_first=True)
# text_field = Field(use_vocab=False, 
#                    tokenize=tokenizer.encode, 
#                    include_lengths=False, 
#                    batch_first=True,
#                    fix_length=MAX_SEQ_LEN, 
#                    pad_token=PAD_INDEX, 
#                    unk_token=UNK_INDEX)

# fields = {'text' : ('text', text_field), 'label' : ('label', label_field)}


# # Read preprocessed CSV into TabularDataset and split it into train, test and valid.
# train_data, valid_data, test_data = TabularDataset(path=f"{data_path}/prep_news.csv", 
#                                                    format='CSV', 
#                                                    fields=fields, 
#                                                    skip_header=False).split(split_ratio=[0.70, 0.2, 0.1], 
#                                                                             stratified=True, 
#                                                                             strata_field='label')

# # Create train and validation iterators.
# train_iter, valid_iter = BucketIterator.splits((train_data, valid_data),
#                                                batch_size=BATCH_SIZE,
#                                                device=device,
#                                                shuffle=True,
#                                                sort_key=lambda x: len(x.titletext), 
#                                                sort=True, 
#                                                sort_within_batch=False)

# # Test iterator, no shuffling or sorting required.
# test_iter = Iterator(test_data, batch_size=BATCH_SIZE, device=device, train=False, shuffle=False, sort=False)


In [ ]:
# # Create sentence and label lists
# sentences = train_df.text.values

# # We need to add special tokens at the beginning and end of each sentence for BERT to work properly
# sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
# labels = train_df.label.values
# tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
# print ("Tokenize the first sentence:")
# print (tokenized_texts[10])

In [ ]:
# import torch
# for text, label in tokenized_train_ds.get_index()

In [ ]:
# shuffled_dataset = dataset.shuffle()
# train_ds, test_ds = Dataset.from_pandas(task1_df, split=['train', 'test'])
# tokenized_dataset = dataset.map(lambda x: tokenizer(x["text"]))

In [ ]:
# train_ds, test_ds = Dataset.from_pandas(task1_df, split=['train', 'test'])